# Text classification with an RNN

This is a text classification that uses recurrent neural network on the IMDB large movie review dataset for sentiment analysis.


_source_: https://www.tensorflow.org/text/tutorials/text_classification_rnn

_Tensorflow Version_: >= 2.6.0


Design of Bidirectional Model                                                         |                                       Design of Stack two or more LSTM layers
:------------------------------------------------------------------------------------:|:-----------------------------------------------------------------------------------------------------:
![bidirectional](https://www.tensorflow.org/text/tutorials/images/bidirectional.png)  |  ![layered_bidirectional](https://www.tensorflow.org/text/tutorials/images/layered_bidirectional.png)

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
tfds.disable_progress_bar()

## Helpers functions

In [ ]:
def plot_history_metric(history, metric):
    """
    Plot metric of model train by epochs history.
    """
    plt.plot(history.history[metric])
    plt.plot(history.history['val_' + metric], '')
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend([metric, 'val_' + metric])

## Setup input pipeline

In [ ]:
dataset, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True, shuffle_files=True)
train_dataset, test_dataset = dataset["train"], dataset["test"]

In [ ]:
train_dataset.element_spec

#### Take one text example and its label. 

In [ ]:
example_text, example_label = None, None
for text, label in train_dataset.take(1):
    example_text = text.numpy()
    example_label = label.numpy()

__Text Example:__

In [ ]:
print(example_text)

__Label Example:__

In [ ]:
print(example_label)

## Shuffle the data for train and create batches of these (text, label) pairs

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
example_batch_texts, example_batch_labels = None, None,
for example, label in train_dataset.take(1):
    example_batch_texts = example.numpy()
    example_batch_labels = label.numpy()

__Batch Texts Example:__

In [ ]:
print(example_batch_texts[:3])

__Batch Labels Example:__

In [ ]:
print(example_batch_labels[:3])

## Create the text encoder

In [ ]:
VOCAB_SIZE = 1000

encoder_layer = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder_layer.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
vocab = np.array(encoder_layer.get_vocabulary())

print(vocab[:15])

In [ ]:
encoded_example = encoder_layer(example)[:3].numpy()

print(encoded_example)

In [ ]:
for n in range(2):
    print("Original: ", example[n].numpy())
    print("--------------------------------")
    print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
    print()

## Create the model

In [ ]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=len(encoder_layer.get_vocabulary()),
    output_dim=64,
    mask_zero=True)

In [ ]:
model = tf.keras.Sequential([
    encoder_layer,
    embedding_layer,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
print([layer.supports_masking for layer in model.layers])

### Predict on a sample text without padding

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')

predictions = model.predict(np.array([sample_text]))
print(predictions[0])

### Predict on a sample text with padding

In [ ]:
padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

### Compile the Keras model to configure the training process

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

## Train the model

In [ ]:
history = model.fit(
    train_dataset, 
    epochs=10,
    validation_data=test_dataset,
    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)

plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

### Prediction on a new sentence

In [ ]:
ample_text = ('The movie was cool. The animation and the graphics were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))

print(predictions)

## Stack more two LSTM Layers

Set _return_sequences=True_ to generate sequences of successive outputs for each timestep (a 3D tensor of shape (batch_size, timesteps, output_features)).

In [ ]:
many_outputs_model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [ ]:
many_outputs_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

In [ ]:
many_outputs_history = model.fit(train_dataset, 
                                 epochs=10,
                                 validation_data_set=test_dataset,
                                 validation_steps=30)

In [ ]:
many_outputs_test_loss, many_outputs_test_acc = many_outputs_history.evaluate(test_dataset)

print('Test Loss:', many_outputs_test_loss)
print('Test Accuracy:', many_outputs_test_acc)

In [ ]:
sample_text = ("The movie was not good. The animation and the graphics were terrible. I would not recommend this movie.")
predictions = model.predict(np.array([sample_text]))
print(predictions)

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plot_graphs(many_outputs_history, 'accuracy')

plt.subplot(1, 2, 2)
plot_graphs(many_outputs_history, 'loss')